In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# digitsデータを使うためにimportする
from torchvision.datasets import MNIST

# digitsデータを使うためにimportする
from sklearn.datasets import load_digits

## GPUを使える場合はGPUを使うための準備

In [ ]:
# GPUを使える場合はGPUを使うための準備
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

## 多層パーセプトロンでの画像分類をPyTorchで実装する際のステップは以下の通り
- step1: データの読み込み
- step2: ネットワークの定義
- step3: 損失関数の定義 
- step4: 最適化関数の定義
- step5: 学習（講義コードはここまで）
- step6: 学習したネットワークを使って未知データの予測

## step1: データの読み込み
---
データを読み込み、Xとy別々に保存

In [ ]:
X = load_digits()["data"]
y = load_digits()["target"]

In [ ]:
idx = 2
plt.imshow(X[idx].reshape(8, 8), cmap=plt.cm.gray_r, interpolation='nearest')
print("Target: ", y[idx])

## NumPyのarrayをtorch.tensor型に変換する
---
torch.tensor型とは、多次元配列を扱うためのデータ構造です。<br>
Numpyのndarrayとほぼ同様の操作ができますが、torch.tensor型はNumpyと違いGPU処理が可能です。

In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(
    X, y, test_size=0.3, random_state=1234)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

In [ ]:
print(X_train_tensor.shape)
print(X_test_tensor.shape)

## TensorDatasetを作成後、DataLoaderを作成
---
TensorDatasetは特徴量Xとラベルyをまとめるデータ構造です。<br>
そのTensorDatasetをDataLoaderに渡すことで、forループでデータの一部のみを受け取れるようにします。

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

In [ ]:
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=True)

## Step2: ネットワークの定義 - nn.Moduleを継承する場合
---
より複雑なネットワークを書くための練習をしましょう。

「Day1_pytorch_intro.ipynb」では、ネットワークの定義を以下のように実行していました。

```
net = nn.Sequential(
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,10)
)
```

しかし上記方法では、分岐を含むような複雑なネットワークを定義することができません。<br>
そのため以下のような書き方を練習しておきましょう！

In [ ]:
# # 「Day1_pytorch_intro.ipynb」の書き方
# net = nn.Sequential(
#     nn.Linear(64,32),
#     nn.ReLU(),
#     nn.Linear(32,16),
#     nn.ReLU(),
#     nn.Linear(16,10)
# )
# net

## ネットワークの書き方 - nn.Moduleを継承する場合

In [ ]:
class myMLP(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.ln1 = nn.Linear(in_features, 32)
        self.ln2 = nn.Linear(32, 16)
        self.ln3 = nn.Linear(16, out_features)

    def forward(self, x):
        x = self.ln1(x)
        x = F.relu(x)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.ln3(x)
        return x

In [ ]:
net = myMLP(64, 10)
net

## Step3: 目的関数に交差エントロピー誤差関数を用意（損失関数の定義）
---
他クラス分類問題なので交差エントロピー誤差関数を設定します。<br>
※ 回帰問題のために、MSELoss()等も用意されています。[Loss functionドキュメント](https://pytorch.org/docs/stable/nn.html#loss-functions)

In [ ]:
loss_func = nn.CrossEntropyLoss()
print(loss_func)

## Step4: Optimizerを定義する（最適化関数の定義
---
今回はAdamを使用してください。<br>
Optimizerには、SGD等も用意されている。[Optimizerドキュメント](https://pytorch.org/docs/stable/optim.html)

In [ ]:
optimizer = optim.Adam(net.parameters())

## GPUに転送
---
GPUはCPUと違って、計算処理を順番に行うのではなく、並行化して行うことができます。<br>
Deep Learningでは膨大な量の行列計算を行うので、GPUを利用し並列化して計算を行うと高速化が見込めます。

In [ ]:
net = net.to(device)

## step5: 学習

In [ ]:
train_loss_track = []
test_loss_track = []

for epoc in range(100):
    running_loss = 0
    
    # 学習モード
    net.train()
    for step, (xx, yy)  in enumerate(train_loader):
        
        # device = "cuda"の場合、GPUにデータを転送する
        xx = xx.to(device)
        yy = yy.to(device)
        
        # 最後に計算した各パラメーターの勾配を初期化する
        optimizer.zero_grad()
        
        # フォワード計算を行う
        y_pred = net(xx)
        
        # 誤差関数を使ってギャップの計測
        loss = loss_func(y_pred, yy)
        
        # 誤差逆伝播法を使って自動微分
        loss.backward()
        
        # パラメーターを更新
        optimizer.step()
        
        # 学習データを使って損失を計算
        running_loss += loss.item()
        
    # エポックが終了したら平均損失を計算
    train_loss_track.append(running_loss/step)
    
    
    #評価（evaluation）モード
    net.eval()
    
    # device = "cuda"であれば評価用データをGPUに転送する
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    
    # 予測値を計算
    y_pred = net(X_test_tensor)
    
    # 交差エントロピー誤差関数を計算
    test_loss = loss_func(y_pred, y_test_tensor)
    
    # 誤差をトラッキング
    test_loss_track.append(test_loss)

In [ ]:
net(X_test_tensor)[0].shape

## 交差エントロピー誤差の推移

In [ ]:
plt.plot(train_loss_track)

In [ ]:
plt.plot(test_loss_track)

## testデータでAccuracyを計算

In [ ]:
true = 0
total = 0

all_labels = np.array([])
all_preds = np.array([])

with torch.no_grad():
    for test_xx, test_yy in test_loader:
        
        # device = "cuda"の場合、GPUにデータを転送する
        test_xx = test_xx.to(device)
        test_yy = test_yy.to(device)

        outputs = net(test_xx)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels = np.append(all_labels, test_yy.cpu().data.numpy())
        all_preds = np.append(all_preds, predicted.cpu().numpy())
        
        total += test_yy.size(0)
        true += (predicted == test_yy).sum().item()
print('Accuracy: {:.2f} %'.format(100 * float(true/total)))

## testデータで混同行列を作成

In [ ]:
labels = np.unique(all_labels)
cm = confusion_matrix(all_labels, all_preds, labels=labels)
cm_labeled = pd.DataFrame(cm, columns=labels, index=labels)
cm_labeled

## 練習１：Dropoutを組み込んでみましょう
---
nn.Dropout(0.5)を組み込んで、オーバーフィッティングしないようにしてみましょう

In [ ]:
# 実装してみる
net = nn.Sequential(
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16,10)
)

In [ ]:
optimizer = optim.Adam(net.parameters())

In [ ]:
net = net.to(device)

train_loss_track = []
test_loss_track = []

for epoc in range(100):
    running_loss = 0
    
    # 学習モード
    net.train()
    for step, (xx, yy)  in enumerate(train_loader):
        
        # device = "cuda"の場合、GPUにデータを転送する
        xx = xx.to(device)
        yy = yy.to(device)
        
        # 最後に計算した各パラメーターの勾配を初期化する
        optimizer.zero_grad()
        
        # フォワード計算を行う
        y_pred = net(xx)
        
        # 誤差関数を使ってギャップの計測
        loss = loss_func(y_pred, yy)
        
        # 誤差逆伝播法を使って自動微分
        loss.backward()
        
        # パラメーターを更新
        optimizer.step()
        
        # 学習データを使って損失を計算
        running_loss += loss.item()
        
    # エポックが終了したら平均損失を計算
    train_loss_track.append(running_loss/step)
    
    
    #評価（evaluation）モード
    net.eval()
    
    # device = "cuda"であれば評価用データをGPUに転送する
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    
    # 予測値を計算
    y_pred = net(X_test_tensor)
    
    # 交差エントロピー誤差関数を計算
    test_loss = loss_func(y_pred, y_test_tensor)
    
    # 誤差をトラッキング
    test_loss_track.append(test_loss)

In [ ]:
plt.plot(train_loss_track)

In [ ]:
plt.plot(test_loss_track)

In [ ]:
# accuracyの計算
true = 0
total = 0

all_labels = np.array([])
all_preds = np.array([])

with torch.no_grad():
    for test_xx, test_yy in test_loader:
        
        # device = "cuda"の場合、GPUにデータを転送する
        test_xx = test_xx.to(device)
        test_yy = test_yy.to(device)

        outputs = net(test_xx)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels = np.append(all_labels, test_yy.cpu().data.numpy())
        all_preds = np.append(all_preds, predicted.cpu().numpy())
        
        total += test_yy.size(0)
        true += (predicted == test_yy).sum().item()
print('Accuracy: {:.2f} %'.format(100 * float(true/total)))

## 練習2：新しいネットワークを書いてみましょう
---
nn.Moduleを継承した書き方で、スライドに記載のネットワークを書いてみましょう。

In [ ]:
class myMLP(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.ln1 = nn.Linear(in_features, 32)
        self.ln2 = nn.Linear(32, 20)
        self.ln3 = nn.Linear(20, 15)
        self.ln4 = nn.Linear(15, out_features)
        
    def forward(self, x):
        x = self.ln1(x)
        x = F.relu(x)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.ln3(x)
        x = F.relu(x)
        x = self.ln4(x)
        return x

In [ ]:
net = myMLP(64, 10)

In [ ]:
optimizer = optim.Adam(net.parameters())

In [ ]:
net = net.to(device)

train_loss_track = []
test_loss_track = []

for epoc in range(100):
    running_loss = 0
    
    # 学習モード
    net.train()
    for step, (xx, yy)  in enumerate(train_loader):
        
        # device = "cuda"の場合、GPUにデータを転送する
        xx = xx.to(device)
        yy = yy.to(device)
        
        # 最後に計算した各パラメーターの勾配を初期化する
        optimizer.zero_grad()
        
        # フォワード計算を行う
        y_pred = net(xx)
        
        # 誤差関数を使ってギャップの計測
        loss = loss_func(y_pred, yy)
        
        # 誤差逆伝播法を使って自動微分
        loss.backward()
        
        # パラメーターを更新
        optimizer.step()
        
        # 学習データを使って損失を計算
        running_loss += loss.item()
        
    # エポックが終了したら平均損失を計算
    train_loss_track.append(running_loss/step)
    
    
    #評価（evaluation）モード
    net.eval()
    
    # device = "cuda"であれば評価用データをGPUに転送する
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    
    # 予測値を計算
    y_pred = net(X_test_tensor)
    
    # 交差エントロピー誤差関数を計算
    test_loss = loss_func(y_pred, y_test_tensor)
    
    # 誤差をトラッキング
    test_loss_track.append(test_loss)

In [ ]:
plt.plot(train_loss_track)

In [ ]:
plt.plot(test_loss_track)

In [ ]:
# accuracyの計算
true = 0
total = 0

all_labels = np.array([])
all_preds = np.array([])

with torch.no_grad():
    for test_xx, test_yy in test_loader:
        
        # device = "cuda"の場合、GPUにデータを転送する
        test_xx = test_xx.to(device)
        test_yy = test_yy.to(device)

        outputs = net(test_xx)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels = np.append(all_labels, test_yy.cpu().data.numpy())
        all_preds = np.append(all_preds, predicted.cpu().numpy())
        
        total += test_yy.size(0)
        true += (predicted == test_yy).sum().item()
print('Accuracy: {:.2f} %'.format(100 * float(true/total)))

## 練習3：MNISTをやってみよう
---
Day1_pytorch_MNIST.ipynbを開けてください。<br>
※データを読み込む部分までは書いてありますので、残りの部分をコーディングしてください。